In [1]:
!pip install confluent-kafka==1.7.0

In [3]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka import KafkaException
import sys
from uuid import uuid4
import random

In [4]:
bootstrap_server = "kafka:9092" # Brokers act as cluster entripoints

In [5]:
conf = {'bootstrap.servers': bootstrap_server}

In [6]:
a = AdminClient(conf)

In [7]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

 1 topics:
  "_schemas" with 1 partition(s)


In [8]:
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import *
import time

topic = "ParticularAdEvent"

def delivery_report(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

In [9]:
producer_conf = {
        'bootstrap.servers': bootstrap_server,
        'key.serializer': StringSerializer('utf_8'),
        'value.serializer': StringSerializer('utf_8')
}

producer = SerializingProducer(producer_conf)

Here we are generating advertisements. They are generated randomly

In [10]:
import json
from random import gauss
from IPython.display import clear_output

keys = ["A", "B", "C", "D", "F", "G"]

i = 0

while True:
    if i == len(keys):
        i = 0
    key = keys[i]
    value = {"ad": key, "ts":int(time.time())}
    producer.produce(topic=topic, value=json.dumps(value), key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    i += 1
    time.sleep(6)
    clear_output(wait=True)


{'ad': 'F', 'ts': 1650970393}
Produced record to topic ParticularAdEvent partition [0] @ offset 178


KeyboardInterrupt: 